# OpenEO Connection to Google Earth Engine

In [1]:
import openeo
from openeo.internal.graph_building import PGNode
import json
import logging
logging.basicConfig(level=logging.INFO)

In [2]:
# Define constants

# Connection
GEE_DRIVER_URL = "https://earthengine.openeo.org/v1.0"
OUTPUT_FILE = "/tmp/openeo_gee_output.png"
OUTFORMAT = "png"

# Data
PRODUCT_ID = "COPERNICUS/S2"

DATE_START = "2017-01-01T00:00:00Z"
DATE_END = "2017-01-31T23:59:59Z"

IMAGE_WEST = 16.138916
IMAGE_EAST = 16.524124
IMAGE_NORTH = 48.320647
IMAGE_SOUTH = 48.138600
IMAGE_SRS = "EPSG:4326"

# Processes
NDVI_RED = "B4"
NDVI_NIR = "B8"

STRECH_COLORS_MIN = -1
STRECH_COLORS_MAX = 1

In [3]:
# Connect with GEE backend and authenticate with basic authentication
con = openeo.connect(GEE_DRIVER_URL)
con.authenticate_basic()
con

<Connection to 'https://earthengine.openeo.org/v1.0' with BearerAuth>

In [4]:
# Get available processes from the back end.
processes = con.list_processes()
processes

[{'categories': ['math'],
  'description': 'Computes the absolute value of a real number `x`, which is the "unsigned" portion of x and often denoted as *|x|*.\n\nThe no-data value `null` is passed through and therefore gets propagated.',
  'examples': [{'arguments': {'x': 0}, 'returns': 0},
   {'arguments': {'x': 3.5}, 'returns': 3.5},
   {'arguments': {'x': -0.4}, 'returns': 0.4},
   {'arguments': {'x': -3.5}, 'returns': 3.5}],
  'id': 'absolute',
  'links': [{'href': 'http://mathworld.wolfram.com/AbsoluteValue.html',
    'rel': 'about',
    'title': 'Absolute value explained by Wolfram MathWorld'}],
  'parameters': [{'description': 'A number.',
    'name': 'x',
    'schema': {'type': ['number', 'null']}}],
  'returns': {'description': 'The computed absolute value.',
   'schema': {'minimum': 0, 'type': ['number', 'null']}},
  'summary': 'Absolute value'},
 {'categories': ['math'],
  'description': 'Sums up the two numbers `x` and `y` (*x + y*) and returns the computed sum.\n\nNo-data 

In [5]:
# Retrieve the list of available collections
collections = con.list_collections()

list(collections)[:2]

[{'description': 'Starting in 2009, the Earth Observation Team of the Science and Technology\nBranch (STB) at Agriculture and Agri-Food Canada (AAFC) began the process\nof generating annual crop type digital maps. Focusing on the Prairie\nProvinces in 2009 and 2010, a Decision Tree (DT) based methodology was\napplied using optical (Landsat-5, AWiFS, DMC) and radar (Radarsat-2) based\nsatellite images. Beginning with the 2011 growing season, this activity has\nbeen extended to other provinces in support of a national crop inventory.\nTo date this approach can consistently deliver a crop inventory that meets\nthe overall target accuracy of at least 85% at a final spatial resolution of\n30m (56m in 2009 and 2010).\n',
  'extent': {'spatial': {'bbox': [[-135.17, 36.83, -51.24, 62.25]]},
   'temporal': {'interval': [['2009-01-01T00:00:00Z', None]]}},
  'id': 'AAFC/ACI',
  'license': 'proprietary',
  'links': [{'href': 'https://earthengine.openeo.org/v1.0/collections/AAFC/ACI',
    'rel': 's

In [6]:
# Get detailed information about a collection
process = con.describe_collection('COPERNICUS/S2')
process

{'cube:dimensions': {'bands': {'type': 'bands',
   'values': ['B1',
    'B2',
    'B3',
    'B4',
    'B5',
    'B6',
    'B7',
    'B8',
    'B8A',
    'B9',
    'B10',
    'B11',
    'B12',
    'QA10',
    'QA20',
    'QA60']},
  't': {'extent': ['2015-06-23T00:00:00Z', None], 'type': 'temporal'},
  'x': {'axis': 'x', 'extent': [-180, 180], 'type': 'spatial'},
  'y': {'axis': 'y', 'extent': [-56, 83], 'type': 'spatial'}},
 'description': 'Sentinel-2 is a wide-swath, high-resolution, multi-spectral\nimaging mission supporting Copernicus Land Monitoring studies,\nincluding the monitoring of vegetation, soil and water cover,\nas well as observation of inland waterways and coastal areas.\n\nThe Sentinel-2 data contain 13 UINT16 spectral bands representing\nTOA reflectance scaled by 10000. See the [Sentinel-2 User Handbook](https://sentinel.esa.int/documents/247904/685211/Sentinel-2_User_Handbook)\nfor details. In addition, three QA bands are present where one\n(QA60) is a bitmask band wi

In [7]:
# Select collection product to get a datacube object
datacube = con.load_collection("COPERNICUS/S2")

In [8]:
# Specifying the date range, the bounding box and bands
datacube = datacube.filter_bbox(west=IMAGE_WEST, east=IMAGE_EAST, north=IMAGE_NORTH,
                                             south=IMAGE_SOUTH, crs=IMAGE_SRS)
datacube = datacube.filter_temporal(DATE_START, DATE_END)

datacube = datacube.filter_bands([NDVI_RED, NDVI_NIR])

print(datacube.to_json())

{'filterbands1': {'arguments': {'bands': ['B4', 'B8'],
   'data': {'from_node': 'filtertemporal1'}},
  'process_id': 'filter_bands',
  'result': True},
 'filterbbox1': {'arguments': {'data': {'from_node': 'loadcollection1'},
   'extent': {'crs': 'EPSG:4326',
    'east': 16.524124,
    'north': 48.320647,
    'south': 48.1386,
    'west': 16.138916}},
  'process_id': 'filter_bbox'},
 'filtertemporal1': {'arguments': {'data': {'from_node': 'filterbbox1'},
   'extent': ['2017-01-01T00:00:00Z', '2017-01-31T23:59:59Z']},
  'process_id': 'filter_temporal'},
 'loadcollection1': {'arguments': {'id': 'COPERNICUS/S2',
   'spatial_extent': None,
   'temporal_extent': None},
  'process_id': 'load_collection'}}

In [9]:
# Applying some operations on the data

# Defining NDVI reducer
red = PGNode("array_element", arguments={"data": {"from_parameter": "data"}, "label": "B4"})
nir = PGNode("array_element", arguments={"data": {"from_parameter": "data"}, "label": "B8"})
ndvi = PGNode("normalized_difference", arguments={"x": {"from_node": nir}, "y": {"from_node": red}})

datacube = datacube.reduce_dimension(dimension="bands", reducer=ndvi)

# take minimum time to reduce by time
datacube = datacube.min_time()

# Linear scale necessary for GEE png export
lin_scale = PGNode("linear_scale_range", arguments={"x": {"from_parameter": "x"}, "inputMin": -1, "inputMax": 1, "outputMax": 255})
datacube = datacube.apply(lin_scale)

# Save result as PNG
datacube = datacube.save_result(format="PNG")

print(datacube.to_json())

{'apply1': {'arguments': {'data': {'from_node': 'reducedimension2'},
   'process': {'process_graph': {'linearscalerange1': {'arguments': {'inputMax': 1,
       'inputMin': -1,
       'outputMax': 255,
       'x': {'from_parameter': 'x'}},
      'process_id': 'linear_scale_range',
      'result': True}}}},
  'process_id': 'apply'},
 'filterbands1': {'arguments': {'bands': ['B4', 'B8'],
   'data': {'from_node': 'filtertemporal1'}},
  'process_id': 'filter_bands'},
 'filterbbox1': {'arguments': {'data': {'from_node': 'loadcollection1'},
   'extent': {'crs': 'EPSG:4326',
    'east': 16.524124,
    'north': 48.320647,
    'south': 48.1386,
    'west': 16.138916}},
  'process_id': 'filter_bbox'},
 'filtertemporal1': {'arguments': {'data': {'from_node': 'filterbbox1'},
   'extent': ['2017-01-01T00:00:00Z', '2017-01-31T23:59:59Z']},
  'process_id': 'filter_temporal'},
 'loadcollection1': {'arguments': {'id': 'COPERNICUS/S2',
   'spatial_extent': None,
   'temporal_extent': None},
  'process_id

In [10]:
# Sending the job to the backend
job = datacube.create_job()
job.start_and_wait().download_results(OUTPUT_FILE)
job

0:00:00.000001 Job 'XfUBRlYFgKe3SBvA': running (progress N/A)
0:00:05.107501 Job 'XfUBRlYFgKe3SBvA': running (progress N/A)
0:00:11.498349 Job 'XfUBRlYFgKe3SBvA': error (progress N/A)


JobFailedException: Batch job XfUBRlYFgKe3SBvA didn't finish properly. Status: error (after 0:00:11.604513).

In [ ]:
# Showing the result
from IPython.display import Image
result = Image(filename=OUTPUT_FILE)

result